## Surnjani Djoko
## Finalized Dec 5, 2021, also incorporated some of Siqi's LOO binning and business logic
## Feature imputation and Feature transformation on test dataset

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Define functions

In [2]:
# show visual count of categorical variable

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def view_categorical_count(df, var):
    
    var_count = df[var].value_counts()
    sns.set(style="darkgrid")
    sns.barplot(var_count.index, var_count.values, alpha=0.9)
    plt.title('Frequency Distribution of {var}')
    plt.ylabel('Number of Occurrences', fontsize=12)
    plt.xlabel(var, fontsize=12)
    plt.show()
    
# provide quick ratio count of categorical variable
#Getting frequency count of a categorical variable and convert it into percentage
def compute_categorical_value_ratio(df, var):
    return df[var].value_counts()/df[var].count()*100

def inspect_missing_value(df):
    #Calculating percentage of missing values per column
    print(df.isna().mean().round(4) * 100)
    #df.isnull().sum()/df.shape[0]*100
    # remove columns that have more than 90% nan rows
    #df = df.loc[:,df.isnull().sum() < 0.9*df.shape[0]]
    #Get the column with the largest number of missing values
    print(df.isnull().sum().nlargest(5))
    
# check any duplicated row
#df.duplicated().any().sum()

In [3]:
# extract categorical vs. numerical columns
def find_diff_type_var(df):
    cat_var = df.loc[:,df.dtypes == np.object].columns
    num_var = df.loc[:,df.dtypes != np.object].columns
    return(cat_var, num_var)

In [4]:
# numerical correlation
def plot_pearson_correlation(df, cols, low_thres, high_thres):
    # customized
    
    corr_df = df[cols].corr()
    # Get only half portion of corr_df to avoid df, so create mask    
    mask = np.triu(np.ones_like(corr_df), k=1)
    
    # plot a heatmap of the values
    plt.figure(figsize=(20,14))
    ax = sns.heatmap(corr_df, vmin=-1, vmax=1, cbar=False,
                     cmap='coolwarm', mask=mask, annot=True)
    
    # format the text in the plot to make it easier to read
    for text in ax.texts:
        t = float(text.get_text())
        if low_thres < t < high_thres:
            text.set_text('')
        else:
            text.set_text(round(t, 2))
        text.set_fontsize('x-large')
    plt.xticks( size='x-large')
    plt.yticks(rotation=0, size='x-large')
    # Save fig uncomment first
    # plt.savefig("Heatmap DF")
    plt.show()
    return corr_df

# in case the table is getting too big, we can use the list that cap the threshold
def list_pearson_correlation(df, cols, low_thres, high_thres):
    '''
    low_thres ~ -0.5
    high_thres ~ 0.5
    '''
    
    corr = df[cols].corr()
    corr_list=[]
    for i in range(0, len(corr.index)):
        for j in range(0, len(corr.columns)):
            if ((corr.iloc[i,j] > high_thres or corr.iloc[i,j] < low_thres) and corr.iloc[i,j] != 1):
                corr_list.append(str(corr.index[i] + ' and ' + corr.columns[j] + ' : ' + str(round(corr.iloc[i,j], 2))))       

    corr_list = '\n'.join(corr_list)
    print(corr_list)

In [5]:
# numerical imputation using related segment with groupby
def impute_grpby(df, col, grpby, impt_fct):
    '''
    col is the column to be imputed
    grpby can be a string or list
    impt_fct provide function of imputation - mean, max, min, median, etc.
    ''' 
    #display(df.groupby(grpby,dropna=False)[col].transform(impt_fct))
    # --------- transform is for individual series????
    df[col] = df[col].fillna(df.groupby(grpby)[col].transform(impt_fct))
    return df[col]

In [6]:
# categorical imputation using WoE
# (1) calculate weight of evidence for each categorical column 
# after reviewing we can decide to use it or not
# https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html

def IV_WoE(data, col, target, show_woe=True):
    
    newDF, woeDF = pd.DataFrame(), pd.DataFrame()
    do = pd.DataFrame({'x': data[col],'y':data[target]})
    
    #display(do)
    d=do.groupby("x", as_index=False, dropna=False).agg({"y":['count','sum']})
    d.columns=['Category','N','Events']
    d['%N'] = d['N']/d.shape[0]
    #display(d)
    d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
    d['Non-Events'] = d['N'] - d['Events']
    d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
    d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
    d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
    d.insert(loc=0, column='Variable', value=col)
    #print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
    temp =pd.DataFrame({"Variable" : [col], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
    newDF=pd.concat([newDF,temp], axis=0)
    woeDF=pd.concat([woeDF,d], axis=0)
    
    #Show WOE Table
    if show_woe == True:
        display(d)
    return newDF, woeDF

## Read and review the training dataset

In [7]:
df_test = pd.read_csv('data/testing_initial.csv')
print(df_test.shape)
df_test.sample(3)

(3508, 107)


,_id_x,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Fire_Under_Control_Time,Ignition_Source,Incident_Numberinc_,Incident_Station_Area,Incident_Ward,Initial_CAD_Event_Type,Intersection,Last_TFS_Unit_Clear_Time,Level_Of_Origin,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Alarm_Timeinc_,TFS_Arrival_Time,TFS_Firefighter_Casualties,_id_y,ID,NAME,ADDRESS,ADDRESS_POINT_ID,ADDRESS_ID,CENTRELINE_ID,MAINT_STAGE,ADDRESS_NUMBER,LINEAR_NAME_FULL,POSTAL_CODE,GENERAL_USE,CLASS_FAMILY_DESC,ADDRESS_ID_LINK,PLACE_NAME,X,Y,LATITUDE,LONGITUDE,WARD_NAME,MUNICIPALITY_NAME,OBJECTID,geometry,fs_Longitude,fs_Latitude,Station_Area,LABEL,DISTANCE_INCIDENT_FIRESTATION,INCIDENT_DATE,DOW,IS_WEEKEND,YEAR,MONTH,MINUTES_TO_ARRIVE,TFS_ARR_DAY,TFS_ALM_DAY,TFS_ARR_HOUR,TFS_ALM_HOUR,MINUTES_TO_LEAVE,INCIDENT_PERIOD_NUM,INCIDENT_PERIOD_CAT,IS_HOLIDAY,IS_HOLIDAY_LAG1,IS_HOLIDAY_LEAD1,IS_HOLIDAY_SEASON,date,rain_v,snow_v,snow_on_ground_v,temperature_avg,relative_humidity_avg,wind_speed_avg,Incident_Numberdemo,TFS_Alarm_Timedemo,Latitude,Longitude,location,FSA,fsa_imputed,density,poor_building_condition,condo_dwelling,Average_Age,Median_Age,male,min_dist_to_hydrant
2031,1949422,81 - Engine Area,NaN,NaN,0,0,5000.0,NaN,NaN,2019-03-02 01:50:00,NaN,01 - Fire,NaN,NaN,NaN,2019-03-02 01:55:00,82 - Vehicle - Mechanical,F19023139,115,16.0,Vehicle Fire - Highway,Duncan Mill Rd / Lesmill Rd,2019-03-02 02:10:19,NaN,99 - Undetermined (formerly 98),1 - Extinguished by fire department,3,11,51 - Mechanical Failure,901 - Automobile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 - Fire with no evidence from street,2019-03-02 01:41:53,2019-03-02 01:47:17,0,14,5,FIRE STATION 115,115 PARKWAY FOREST DR,8823994,286309,8823988,REGULAR,115,Parkway Forest Dr,NaN,Fire/Ambulance Stations,"Land, Structure, Structure Entrance",NaN,"Fire Station 115, Ambulance Station 23",NaN,NaN,NaN,NaN,Don Valley North (17),North York,1783376,"{u'type': u'Point', u'coordinates': (-79.34068...",-79.340686,43.774404,115,0,13688.244094,2019-03-02 00:00:00,5,1,2019,3,5.400,2,2,1,1,23.033,1,Late Night,0,0,0,0,2019-03-02,1.03,0.89,5.3,-2.950,70.340,18.26,F19023139,2019-03-02 01:41:53,43.76065,-79.358496,M3B 2S7,M3B,0,2203.38,0.039157,0.210843,44.7,47.1,0.477102,19.676431
2997,1963313,"44 - Trash, Rubbish Storage (inc garbage chute...",NaN,NaN,0,0,100.0,NaN,1.0,2012-03-08 03:30:00,NaN,01 - Fire,NaN,NaN,NaN,2012-03-08 03:31:50,999 - Undetermined,F12025521,341,17.0,FIR,Dynevor Rd / Rowan Ave,2012-03-08 03:59:02,NaN,"46 - Rubbish, Trash, Waste",1 - Extinguished by fire department,9,30,45 - Improperly Discarded,848 - Trash/Rubbish/Garbage Container or Dumpster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"7 - Fully involved (total structure, vehicle, ...",2012-03-08 03:24:06,2012-03-08 03:28:48,0,18,59,FIRE STATION 341,555 OAKWOOD AVE,69771,96972,7809,REGULAR,555,Oakwood Ave,NaN,Fire Station,"Land, Structure, Structure Entrance",NaN,Fire Station 341,NaN,NaN,NaN,NaN,Toronto-St. Paul's (12),York,1961768,"{u'type': u'Point', u'coordinates': (-79.44114...",-79.441141,43.694520,341,0,13691.776099,2012-03-08 00:00:00,3,0,2012,3,4.700,8,8,3,3,30.233,1,Late Night,0,0,0,0,2012-03-08,1.10,0.79,5.1,-1.915,71.630,17.84,F12025521,2012-03-08 03:24:06,43.69196,-79.449730,M6E 2J5,M6E,0,7241.08,0.090047,0.011844,41.3,40.8,0.483044,34.647428
2914,1957275,"44 - Trash, Rubbish Storage (inc garbage chute...",NaN,NaN,0,0,100.0,NaN,NaN,2015-07-20 05:01:30,NaN,01

In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3508 entries, 0 to 3507
Columns: 107 entries, _id_x to min_dist_to_hydrant
dtypes: float64(29), int64(30), object(48)
memory usage: 2.9+ MB


In [9]:
inspect_missing_value(df_test)

_id_x                                                              0.00
Area_of_Origin                                                    10.97
Building_Status                                                   36.03
Business_Impact                                                   36.06
Civilian_Casualties                                                0.00
Count_of_Persons_Rescued                                           0.00
Estimated_Dollar_Loss                                             10.95
Estimated_Number_Of_Persons_Displaced                             36.06
Exposures                                                         98.00
Ext_agent_app_or_defer_time                                       10.97
Extent_Of_Fire                                                    36.06
Final_Incident_Type                                                0.00
Fire_Alarm_System_Impact_on_Evacuation                            36.06
Fire_Alarm_System_Operation                                     

In [10]:
cat_cols, num_cols = find_diff_type_var(df_test)
print(cat_cols)
print(num_cols)

Index(['Area_of_Origin', 'Building_Status', 'Business_Impact',
       'Ext_agent_app_or_defer_time', 'Extent_Of_Fire', 'Final_Incident_Type',
       'Fire_Alarm_System_Impact_on_Evacuation', 'Fire_Alarm_System_Operation',
       'Fire_Alarm_System_Presence', 'Fire_Under_Control_Time',
       'Ignition_Source', 'Incident_Numberinc_', 'Initial_CAD_Event_Type',
       'Intersection', 'Last_TFS_Unit_Clear_Time', 'Level_Of_Origin',
       'Material_First_Ignited', 'Method_Of_Fire_Control', 'Possible_Cause',
       'Property_Use', 'Smoke_Alarm_at_Fire_Origin',
       'Smoke_Alarm_at_Fire_Origin_Alarm_Failure',
       'Smoke_Alarm_at_Fire_Origin_Alarm_Type',
       'Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation',
       'Smoke_Spread', 'Sprinkler_System_Operation',
       'Sprinkler_System_Presence', 'Status_of_Fire_On_Arrival',
       'TFS_Alarm_Timeinc_', 'TFS_Arrival_Time', 'NAME', 'ADDRESS',
       'MAINT_STAGE', 'LINEAR_NAME_FULL', 'POSTAL_CODE', 'GENERAL_USE',
       'CL

## Decided in training which columns to drop for the obvious reasons

In [11]:
# identifies columns I used and added that are not needed for prediction, and high null values, ID related ...

my_not_needed_cols = ['Station_Area', 'TFS_Alarm_Timeinc_', 'TFS_Arrival_Time',
                      'YEAR','TFS_ARR_DAY','TFS_ALM_DAY','TFS_ARR_HOUR','TFS_ALM_HOUR']
# additional numerical columns that are not needed
add_num_cols = ['Incident_Station_Area', 'Incident_Ward','ADDRESS_POINT_ID', 'ADDRESS_ID',
       'CENTRELINE_ID', 'ADDRESS_NUMBER', 'OBJECTID', 'fs_Longitude',
       'fs_Latitude', 'Latitude', 'Longitude']
add_cat_cols = ['Intersection', 'Last_TFS_Unit_Clear_Time','TFS_Alarm_Timeinc_', 
        'TFS_Arrival_Time', 'NAME', 'ADDRESS','LINEAR_NAME_FULL', 'PLACE_NAME', 
        'WARD_NAME', 'MUNICIPALITY_NAME', 'geometry','INCIDENT_DATE',
                'Ext_agent_app_or_defer_time','Fire_Under_Control_Time','location']

fire_not_needed_cols = ['_id_x','_id_y','Exposures']
    #Exposure has more than 90% missing
#Demographics_not_needed_cols = ['fsa_imputed','location','FSA'] 
Demographics_not_needed_cols = ['fsa_imputed','location']  
station_not_needed_cols=['ID', 'ADDRESS_ID_LINK', 'X','Y', 'LATITUDE', 'LONGITUDE','POSTAL_CODE','MAINT_STAGE','GENERAL_USE','CLASS_FAMILY_DESC', 'INCIDENT_PERIOD_CAT']
weather_not_needed_cols=['date', 'Incident_Numberdemo','TFS_Alarm_Timedemo']
drop_columns = my_not_needed_cols + add_num_cols + add_cat_cols + Demographics_not_needed_cols + \
    fire_not_needed_cols + station_not_needed_cols + weather_not_needed_cols
df_test.drop(columns=drop_columns, axis=1,inplace=True)
df_test.shape

(3508, 57)

In [12]:
# review after the drop!!
cat_cols, num_cols = find_diff_type_var(df_test)
print(cat_cols)
print(num_cols)

Index(['Area_of_Origin', 'Building_Status', 'Business_Impact',
       'Extent_Of_Fire', 'Final_Incident_Type',
       'Fire_Alarm_System_Impact_on_Evacuation', 'Fire_Alarm_System_Operation',
       'Fire_Alarm_System_Presence', 'Ignition_Source', 'Incident_Numberinc_',
       'Initial_CAD_Event_Type', 'Level_Of_Origin', 'Material_First_Ignited',
       'Method_Of_Fire_Control', 'Possible_Cause', 'Property_Use',
       'Smoke_Alarm_at_Fire_Origin',
       'Smoke_Alarm_at_Fire_Origin_Alarm_Failure',
       'Smoke_Alarm_at_Fire_Origin_Alarm_Type',
       'Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation',
       'Smoke_Spread', 'Sprinkler_System_Operation',
       'Sprinkler_System_Presence', 'Status_of_Fire_On_Arrival', 'FSA'],
      dtype='object')
Index(['Civilian_Casualties', 'Count_of_Persons_Rescued',
       'Estimated_Dollar_Loss', 'Estimated_Number_Of_Persons_Displaced',
       'Number_of_responding_apparatus', 'Number_of_responding_personnel',
       'TFS_Firefighter

## Numerical features

In [13]:
inspect_missing_value(df_test[num_cols])
# only two variables need to be imputed using training information

Civilian_Casualties                       0.00
Count_of_Persons_Rescued                  0.00
Estimated_Dollar_Loss                    10.95
Estimated_Number_Of_Persons_Displaced    36.06
Number_of_responding_apparatus            0.00
Number_of_responding_personnel            0.00
TFS_Firefighter_Casualties                0.00
LABEL                                     0.00
DISTANCE_INCIDENT_FIRESTATION             0.00
DOW                                       0.00
IS_WEEKEND                                0.00
MONTH                                     0.00
MINUTES_TO_ARRIVE                         0.00
MINUTES_TO_LEAVE                          0.00
INCIDENT_PERIOD_NUM                       0.00
IS_HOLIDAY                                0.00
IS_HOLIDAY_LAG1                           0.00
IS_HOLIDAY_LEAD1                          0.00
IS_HOLIDAY_SEASON                         0.00
rain_v                                    0.00
snow_v                                    0.00
snow_on_groun

In [14]:
df_test[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
Civilian_Casualties,3508.0,0.075257,0.336935,0.000000,0.000000,0.000000,0.000000,6.000000e+00
Count_of_Persons_Rescued,3508.0,0.060718,1.489251,0.000000,0.000000,0.000000,0.000000,8.600000e+01
Estimated_Dollar_Loss,3124.0,34417.956786,234126.953281,0.000000,200.000000,2500.000000,15000.000000,1.000000e+07
Estimated_Number_Of_Persons_Displaced,2243.0,16.649576,117.184729,0.000000,0.000000,0.000000,1.000000,9.970000e+02
Number_of_responding_apparatus,3508.0,6.973774,8.351789,1.000000,2.000000,6.000000,8.000000,1.750000e+02
Number_of_responding_personnel,3508.0,23.113740,25.915744,2.000000,8.000000,21.000000,26.000000,5.370000e+02
TFS_Firefighter_Casualties,3508.0,0.014823,0.146462,0.000000,0.000000,0.000000,0.000000,3.000000e+00
LABEL,3508.0,0.059863,0.237267,0.000000,0.000000,0.000000,0.000000,1.000000e+00
DISTANCE_INCIDENT_FIRESTATION,3508.0,13689.263078,10.880229,13667.666323,13680.922331,13687.480410,13697.082392,1.371846e+04
DOW,3508.0,3.022235,1.995309,0.000000,1.000000,3.000000,5.000000,6.000000e+00


## (1) impute 'Estimated_Number_Of_Persons_Displaced'

In [15]:
df_test.Estimated_Number_Of_Persons_Displaced.isnull().sum()
    # there are some missing

1265

In [16]:
# read imputation table that was saved during training
est_mean_df = pd.read_csv('imputation/est_n_person_displaced_mean_df.csv')
print(est_mean_df.shape)
est_mean_df.sample(3)

(38, 3)


,Count_of_Persons_Rescued,Civilian_Casualties,Estimated_Number_Of_Persons_Displaced_mean
10,1,4,997.0
23,5,0,78.0
2,0,2,27.0


In [17]:
# join with the table to extract the imputed value
col = 'Estimated_Number_Of_Persons_Displaced'
grpby=['Count_of_Persons_Rescued','Civilian_Casualties']
df_test = pd.merge(df_test, est_mean_df, how='left', on=grpby)

df_test[col+'_imputed'] = df_test[col]
df_test[col+'_imputed'].fillna(df_test['Estimated_Number_Of_Persons_Displaced_mean'], inplace=True)
#df_test.rename(columns={'Estimated_Number_Of_Persons_Displaced_mean':col+'_imputed'}, inplace=True)

In [18]:
df_test.Estimated_Number_Of_Persons_Displaced_imputed.isnull().sum()
    # there are still some missing

0

In [19]:
df_test.drop('Estimated_Number_Of_Persons_Displaced_mean',inplace=True,axis=1)

## (2) impute 'Estimated_Dollar_Loss'

In [20]:
df_test.Estimated_Dollar_Loss.isnull().sum()
    # there are some missing

384

In [21]:
# this imputation depends on Property_Use, and Property_Use does not have any null, so we are good
# read imputation table that was saved during training
est_dollar_loss_mean_df = pd.read_csv('imputation/est_dollar_loss_mean.csv')
print(est_dollar_loss_mean_df.shape)
est_dollar_loss_mean_df.sample(3)

(3312, 3)


,FSA,Property_Use_imputed,Estimated_Dollar_Loss_mean
1638,M4X,322 - Multi-Unit Dwelling - 7 to 12 Units,20.0
2232,M6C,"896 - Sidewalk, street, roadway, highway, hwy ...",3300.0
2275,M6E,842 - Fence,295.0


In [22]:
# join with the table to extract the imputed value
col = 'Estimated_Dollar_Loss'
grpby=['FSA','Property_Use_imputed']
df_test['Property_Use_imputed'] = df_test['Property_Use']
    # although nothing missing, still copy over in order to be consistent with training dataset
df_test = pd.merge(df_test, est_dollar_loss_mean_df, how='left', on=grpby)

df_test[col+'_imputed'] = df_test[col]
df_test[col+'_imputed'].fillna(df_test['Estimated_Dollar_Loss_mean'], inplace=True)   

In [23]:
df_test.Estimated_Dollar_Loss_imputed.isnull().sum()
    # there are some still missing

82

In [24]:
# a value was computed during training: 33495.76328719448
dollar_loss = 33495.76
df_test.Estimated_Dollar_Loss_imputed.fillna(dollar_loss, inplace=True)

In [25]:
df_test.drop('Estimated_Dollar_Loss_mean',inplace=True,axis=1)

In [26]:
df_test.Estimated_Dollar_Loss_imputed.isnull().sum()
    # there are some still missing

0

In [27]:
inspect_missing_value(df_test)

Area_of_Origin                                                   10.97
Building_Status                                                  36.03
Business_Impact                                                  36.06
Civilian_Casualties                                               0.00
Count_of_Persons_Rescued                                          0.00
Estimated_Dollar_Loss                                            10.95
Estimated_Number_Of_Persons_Displaced                            36.06
Extent_Of_Fire                                                   36.06
Final_Incident_Type                                               0.00
Fire_Alarm_System_Impact_on_Evacuation                           36.06
Fire_Alarm_System_Operation                                      36.06
Fire_Alarm_System_Presence                                       36.06
Ignition_Source                                                  10.97
Incident_Numberinc_                                               0.00
Initia

In [28]:
# copy over some cols to be consistent with training, although they are not missing
df_test['min_dist_to_hydrant_imputed'] = df_test['min_dist_to_hydrant']
df_test['DISTANCE_INCIDENT_FIRESTATION_imputed'] = df_test['DISTANCE_INCIDENT_FIRESTATION']

## Categorical features

In [29]:
df_test['Level_Of_Origin_impute']=df_test['Level_Of_Origin'].fillna('999')

In [30]:
df_test['Level_Of_Origin_impute'].isnull().sum()

0

In [31]:
df_test['Level_Of_Origin_impute'].unique()

array(['999', 'B01', '013', '003', '998', '005', '001', 'B02', '028',
       '002', '017', '004', '022', '012', '006', '020', '009', '047',
       '007', '996', '027', '997', '016', '008', '015', '018', '014',
       '030', '011', '010', 'B09', 'B03', 'B04', '024', '025', '021',
       '019', '029', '039', '035', '042', '023', '034', '026'],
      dtype=object)

In [32]:
#Level_of_Origin: https://www.toronto.ca/ext/open_data/catalog/data_set_files/ofmcodes2009.pdf
LOO = df_test['Level_Of_Origin']
LOO_encoded = [None]*len(LOO)
for i in range(len(LOO)):
    try: 
        if int(LOO[i])<=3:
            LOO_encoded[i]='low_rise_level'
        elif int(LOO[i])<=12:
            LOO_encoded[i]='mid_rise_level'
        elif int(LOO[i])<=100:
            LOO_encoded[i]='high_rise_level'
        elif int(LOO[i])==996:
            LOO_encoded[i]='roof'
        else:
            LOO_encoded[i]='other'
    except: # this will start with 'B'
        LOO_encoded[i]='below_grade_level'
print(LOO_encoded[0:50])
df_test['Level_Of_Origin_encoded']=LOO_encoded

['below_grade_level', 'below_grade_level', 'high_rise_level', 'low_rise_level', 'low_rise_level', 'other', 'below_grade_level', 'mid_rise_level', 'low_rise_level', 'below_grade_level', 'below_grade_level', 'below_grade_level', 'low_rise_level', 'low_rise_level', 'below_grade_level', 'low_rise_level', 'below_grade_level', 'below_grade_level', 'high_rise_level', 'below_grade_level', 'low_rise_level', 'low_rise_level', 'low_rise_level', 'high_rise_level', 'low_rise_level', 'below_grade_level', 'below_grade_level', 'below_grade_level', 'low_rise_level', 'below_grade_level', 'low_rise_level', 'low_rise_level', 'low_rise_level', 'below_grade_level', 'mid_rise_level', 'high_rise_level', 'below_grade_level', 'low_rise_level', 'low_rise_level', 'low_rise_level', 'mid_rise_level', 'below_grade_level', 'below_grade_level', 'below_grade_level', 'below_grade_level', 'below_grade_level', 'low_rise_level', 'other', 'low_rise_level', 'below_grade_level']


In [33]:
df_test['Level_Of_Origin_encoded'].head(10)

0    below_grade_level
1    below_grade_level
2      high_rise_level
3       low_rise_level
4       low_rise_level
5                other
6    below_grade_level
7       mid_rise_level
8       low_rise_level
9    below_grade_level
Name: Level_Of_Origin_encoded, dtype: object

In [34]:
# impute with appropriate logic
def impute_with_business_logic(df):
    df.at[df[df['Smoke_Alarm_at_Fire_Origin_Alarm_Failure'].isnull()].index,'Smoke_Alarm_at_Fire_Origin_Alarm_Failure']='98 - Not applicable: Alarm operated OR presence/operation undetermined'
    df.at[df[df['Status_of_Fire_On_Arrival'].isnull()].index,'Status_of_Fire_On_Arrival']='Not applicable'
    df.at[df[df['Smoke_Spread'].isnull()].index,'Smoke_Spread']='Not applicable'
    df.at[df[df['Sprinkler_System_Operation'].isnull()].index,'Sprinkler_System_Operation']='8 - Not applicable - no sprinkler system present'
    df.at[df[df['Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation'].isnull()].index,'Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation']='Not applicable'
    df.at[df[df['Business_Impact'].isnull()].index,'Business_Impact']='8 - Not applicable (not a business)'
    df.at[df[df['Fire_Alarm_System_Impact_on_Evacuation'].isnull()].index,'Fire_Alarm_System_Impact_on_Evacuation']='Not applicable'
    df.at[df[df['Smoke_Alarm_at_Fire_Origin_Alarm_Type'].isnull()].index,'Smoke_Alarm_at_Fire_Origin_Alarm_Type']='8 - Not applicable - no smoke alarm or presence undetermined'
    df.at[df[df['Smoke_Alarm_at_Fire_Origin'].isnull()].index,'Smoke_Alarm_at_Fire_Origin']='Not applicable'
    df.at[df[df['Fire_Alarm_System_Operation'].isnull()].index,'Fire_Alarm_System_Operation']='8 - Not applicable (no system)'
    df.at[df[df['Sprinkler_System_Presence'].isnull()].index,'Sprinkler_System_Presence']='9 - Undetermined'
    df.at[df[df['Fire_Alarm_System_Presence'].isnull()].index,'Fire_Alarm_System_Presence']='2 - No Fire alarm system'
    df.at[df[df['Building_Status'].isnull()].index,'Building_Status']='08 - Not Applicable'
    return(df)

In [35]:
df_test = impute_with_business_logic(df_test)

In [36]:
# --------- IMPUTATION and transformation at the same time -------
# categorical imputation using WoE, need to review each individual col if it is appropriate
# fillna to 'Unknown', and create each WoE table for each category
# put into one big dictionary for future access!!

# from the training:
# Extent_Of_Fire is fillna with '8 - Entire Structure'

In [37]:
df_test['Extent_Of_Fire'+'_imputed'] = df_test['Extent_Of_Fire']
df_test['Extent_Of_Fire_imputed'].fillna('8 - Entire Structure', inplace=True)

In [38]:
# perform the impute first
cols4WoE_ = ['Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation','Smoke_Spread',                                                                     
    'Level_Of_Origin_encoded'] 
for cat in cols4WoE_:
    df_test[cat+'_imputed'] = df_test[cat]
    df_test[cat+'_imputed'].fillna('Unknown', inplace=True)

In [39]:
# perform WoE
cols4WoE = ['Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation','Smoke_Spread',                                                                     
    'Level_Of_Origin_encoded','Extent_Of_Fire'] 
for cat in cols4WoE:
    # going through each categorical variable
    print(cat)
    woe_pd = pd.read_csv(cat+'_woe.csv')
    # apply the woe
    woe_map = dict(zip(woe_pd.Category, woe_pd.WoE))
    print(woe_map)
    df_test[cat+'_woe'] = df_test[cat+'_imputed'].map(woe_map)

Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation
{'4 - Alarm operated but failed to alert occupant(s) (at risk)': 2.565878441247876, '2 - Some persons (at risk) self evacuated as a result of hearing alarm': 1.1607979740069103, '1 - All persons (at risk of injury) self evacuated as a result of hearing alarm': 0.6726431632906664, '9 - Undetermined': 0.5579027121381774, '7 - Not applicable: Occupant(s) first alerted by other means': 0.3609805199070757, '3 - No one (at risk) evacuated as a result of hearing alarm': 0.2632933482538301, '8 - Not applicable: No alarm, no persons present': -1.1426207997357127, 'Not applicable': -2.1972314409661875}
Smoke_Spread
{'5 - Multi unit bldg: spread beyond suite of origin but not to separated suite(s)': 1.160535885157291, '6 - Multi unit bldg: spread to separate suite(s)': 1.0753230698035758, '7 - Spread to other floors, confined to building': 0.8446829357388135, '8 - Entire Structure': 0.7692289867256292, '4 - Spread beyond room of origin

In [40]:
inspect_missing_value(df_test)

Area_of_Origin                                                           10.97
Building_Status                                                           0.00
Business_Impact                                                           0.00
Civilian_Casualties                                                       0.00
Count_of_Persons_Rescued                                                  0.00
Estimated_Dollar_Loss                                                    10.95
Estimated_Number_Of_Persons_Displaced                                    36.06
Extent_Of_Fire                                                           36.06
Final_Incident_Type                                                       0.00
Fire_Alarm_System_Impact_on_Evacuation                                    0.00
Fire_Alarm_System_Operation                                               0.00
Fire_Alarm_System_Presence                                                0.00
Ignition_Source                                     

In [41]:
df_test.to_csv('data/test_impute_transform.csv', index=False)

In [42]:
df_test.shape

(3508, 72)